# Combine Wiki Dataset and Law Dataset

In [20]:
import pandas as pd
from tokenizers import Tokenizer
from tokenizers.processors import TemplateProcessing
from tokenizers.normalizers import (Sequence, NFD)
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.models import BPE
from tokenizers.decoders import BPEDecoder
from tokenizers.trainers import BpeTrainer

In [21]:
train_law_par = pd.read_parquet(r"c:\Users\siren\Desktop\1507\dataset\hukuk_raw_dataset\train.parquet", engine="pyarrow")
wiki_par = pd.read_parquet(r"C:\Users\siren\Desktop\1507\dataset\ts_wikipedia\bpe_support.parquet", engine="pyarrow")

In [22]:
wiki_par = wiki_par.drop([2372009])
train_law_par=train_law_par.drop(columns = ["source"])

In [23]:
all_df = pd.concat([wiki_par,train_law_par])

In [24]:
all_df.reset_index(drop=True,inplace=True)

In [25]:
all_df

,text
0,Cengiz Han Moğol Börçigin ailesinden siyasetçi...
1,Moğol kabilelerini buyruğu altında birleştirer...
2,Bu imparatorluk Dünya tarihinin bitişik sınırl...
3,Timuçin ismiyle Moğol Devleti hükümdarı akraba...
4,Tüm dünya tarafından genelde acımasız ve kana ...
...,...
5396531,uzatmayı öngören hükümlere sözleşmede yer verm...
5396532,163 Tam yargı davası 2577 sayılı İdari Yargıla...
5396533,sürdürmesinden ibaret ve bu hakkın kişinin öze...
5396534,2. Portföy Yönetim Şirketleri ve Bireysel Katı...


In [26]:
special_tokens = ["[UNK]", "<|endoftext|>"]

tokenizer = Tokenizer(BPE(unk_token="[UNK]"))
tokenizer.normalizer = Sequence([NFD()])
tokenizer.pre_tokenizer = Whitespace()
tokenizer.decoder = BPEDecoder()
tokenizer.post_processor = TemplateProcessing(single="$A <|endoftext|>",  
                                              special_tokens=[("<|endoftext|>", 1)]  
                                              )

In [27]:
trainer = BpeTrainer(vocab_size=52000,
                     min_frequency = 1,
                     special_tokens=special_tokens,
                     continuing_subword_prefix="##",
                     max_token_length = 6,
                     show_progress = True)
tokenizer.train_from_iterator(all_df["text"].astype(str), trainer=trainer)

In [28]:
tokenizer.model.save('.')
tokenizer.save("MyBPETokenizerWikiLaw.json")

In [29]:
tokenizerFromFile = Tokenizer.from_file("MyBPETokenizerWikiLaw.json")

In [30]:
sen_enc3 = tokenizerFromFile.encode("Hukuk bir gün herkese lazım olur.")
print(f"Output: {format(sen_enc3.tokens)}")

Output: ['Hukuk', 'bir', 'gün', 'herk', '##ese', 'lazım', 'olur', '.', '<|endoftext|>']


In [31]:
sen_enc3 = tokenizerFromFile.encode("Bırakın adalet yerini bulsun, isterse kıyamet kopsun.")
print(f"Output: {format(sen_enc3.tokens)}")

Output: ['Bırak', '##ın', 'ada', '##let', 'yer', '##ini', 'bul', '##sun', ',', 'ister', '##se', 'kıya', '##met', 'kop', '##sun', '.', '<|endoftext|>']


In [32]:
sen_enc3 = tokenizerFromFile.encode("Adalet önce devletten gelir. Çünkü hukuk, devletin toplumsal düzenidir.")
print(f"Output: {format(sen_enc3.tokens)}")

Output: ['Ada', '##let', 'önce', 'dev', '##let', '##ten', 'gelir', '.', 'Çün', '##kü', 'hukuk', ',', 'dev', '##letin', 'top', '##lum', '##sal', 'düz', '##eni', '##dir', '.', '<|endoftext|>']


In [56]:
tokenizerFromFile.encode(train_law_par.iloc[0]["text"])

Encoding(num_tokens=664, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])